#### Run the following cell to import necessary packages

In [11]:
from __future__ import print_function
from random import randint
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from traitlets import traitlets

import pandas as pd
import numpy as np
import ipywidgets

import matplotlib.pyplot as plt

%matplotlib inline

#### Running the following cell will instantiate the functions for the program to work

In [3]:
def compute_vol(trading_period_digits_list):
    period_vols = []
    for i in trading_period_digits_list:
        single_period_vol = (i - 4)**2
        period_vols.append(single_period_vol)
    return sum(period_vols)

def create_teams_list():
    team_names = []
    while True:
        team = input("Enter a team name (Type 'q' to quit): ")
        if team == 'q':
            break
        team_names.append(team)
    return team_names

def create_manual_digits_list():
    digits = []
    while True:
        digit = input("Enter each periods trading digit (Type 'q' to quit when finished): ")
        if digit == 'q':
            break
        digits.append(int(digit))
    return digits

def produce_rand_int():
    return randint(0,9)

def create_auto_digits_list():
    auto_digits = []
    for i in range(7):
        auto_digits.append(produce_rand_int())
    return auto_digits

#### Run the following cell, input each team's name. If a typo is made, just re-run the cell.

In [4]:
teams = create_teams_list()

Enter a team name (Type 'q' to quit): team_a
Enter a team name (Type 'q' to quit): team_b
Enter a team name (Type 'q' to quit): team_c
Enter a team name (Type 'q' to quit): q


#### Run the following cell, input appropriate responses, and click submit trade to log a new trade. Change responses and click submit again to log each new trade.

In [37]:
input_trade_accordion = widgets.Accordion(children=[widgets.Select(options=teams,disabled=False),
                                                    widgets.Select(options=teams,disabled=False),
                                                    widgets.FloatText()])
input_trade_accordion.set_title(0, 'Buyer')
input_trade_accordion.set_title(1, 'Seller')
input_trade_accordion.set_title(2, 'Price')

class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

def add_trades(b):
    b.value = b.value.append({'buyer':input_trade_accordion.children[0].value,
                              'seller':input_trade_accordion.children[1].value,
                              'price':input_trade_accordion.children[2].value}, ignore_index=True)

lb = LoadedButton(description="Click to Submit Trade", value=pd.DataFrame(columns=['buyer','seller','price']))
lb.on_click(add_trades)
display(input_trade_accordion, lb)

Accordion(children=(Select(options=('team_a', 'team_b', 'team_c'), value='team_a'), Select(options=('team_a', …

LoadedButton(description='Click to Submit Trade', style=ButtonStyle())

#### Run the next cell to create trade_log variable

In [40]:
#Assign values from input to trade_log variable
trade_log = lb.value

#### Run the following cell to take a look at the trade log

In [41]:
trade_log

,buyer,seller,price
0,team_a,team_b,3.0
1,team_b,team_a,6.0
2,team_b,team_a,7.0
3,team_a,team_c,12.0


#### Run following cell, input the digits for the trading periods. If a typo is made, just re-run the cell

In [42]:
trading_digits = create_manual_digits_list()

Enter each periods trading digit (Type 'q' to quit when finished): 1
Enter each periods trading digit (Type 'q' to quit when finished): 6
Enter each periods trading digit (Type 'q' to quit when finished): 4
Enter each periods trading digit (Type 'q' to quit when finished): 5
Enter each periods trading digit (Type 'q' to quit when finished): 7
Enter each periods trading digit (Type 'q' to quit when finished): 8
Enter each periods trading digit (Type 'q' to quit when finished): 9
Enter each periods trading digit (Type 'q' to quit when finished): q


#### Running the following cell will compute the P/L for each team

In [45]:
#Compute volatility across all periods, append to trade_log df
trade_log['contract_value'] = compute_vol(trading_digits)

#compute spreads for buyers and sellers on each trade
trade_log['buy_spread'] = trade_log['contract_value'] - trade_log['price']
trade_log['sell_spread'] = trade_log['price'] - trade_log['contract_value']

#compute team P/L into 2 dataframes for buyers, and sellers
buy_pl = trade_log.groupby('buyer')['buy_spread'].sum().to_frame()
sell_pl = trade_log.groupby('seller')['sell_spread'].sum().to_frame()

#join the dataframes, rename index, compute total P/L
combined_pl = buy_pl.join(sell_pl)
combined_pl.index.names = ['team']
combined_pl['P/L'] = combined_pl['buy_spread'] + combined_pl['sell_spread']

#display P/L
combined_pl

,buy_spread,sell_spread,P/L
team,,,
team_a,113.0,-115.0,-2.0
team_b,115.0,-61.0,54.0


In [47]:
trade_log

,buyer,seller,price,contract_value,buy_spread,sell_spread
0,team_a,team_b,3.0,64,61.0,-61.0
1,team_b,team_a,6.0,64,58.0,-58.0
2,team_b,team_a,7.0,64,57.0,-57.0
3,team_a,team_c,12.0,64,52.0,-52.0
